In [4]:
import pandas as pd
import numpy as np
import re
import string
from collections import Counter
import csv
import math
import nltk
from nltk.stem import SnowballStemmer 
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split

**Reading the csv file**

In [2]:
#Creating a data frame out of the csv file
df = pd.read_csv(r"C:\Users\91971\Desktop\organized\python\Machine Learning codes\Sentiment analysis of amazon product reviews\amazon data\1429_1.csv")
df.head() #head function is used to show the top 5 rows of dataframe

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\91971\\Desktop\\organized\\python\\Machine Learning codes\\Sentiment analysis of amazon product reviews\\amazon data\\1429_1.csv'

In [3]:
#Some information about the dataset
df.info()

NameError: name 'df' is not defined

In [61]:
df["reviews.rating"].value_counts().sort_values(ascending = False)

reviews.rating
5.0    23775
4.0     8541
3.0     1499
1.0      410
2.0      402
Name: count, dtype: int64

**Converting the emojis to text**

In [64]:
#Converting Emojis to their Respective Emotions
df["reviews.text"] = df["reviews.text"].replace([r"\:\)",r"\:\-\)", r"\:\-\}",r"\;\-\}",r"\:\-\>",r"\;\-\)"], ["Happy","Happy","Happy","Happy","Happy","Happy"], regex=True)
df["reviews.text"] = df["reviews.text"].replace([r"\:\-\(",r"\:\(",r"\:\-\|",r"\;\-\(",r"\;\-\<",r"\|\-\{"], ["Sad", "Sad", "Sad", "Sad", "Sad", "Sad",], regex=True)
df["reviews.text"] = df["reviews.text"].replace([r"\:\D",r"\:\'\-\)",r"\:\`\-\("], ["laugh", "tear of joy", "tear of sadness"], regex=True)

In [66]:

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Initialize stemmer and stopwords
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91971\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91971\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [100]:
# Text preprocessing function
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    words = word_tokenize(text)
    # Remove stopwords and stem
    words = [stemmer.stem(w) for w in words if w not in stop_words and len(w) > 2]
    return ' '.join(words)

In [70]:
data_set = df[["reviews.text", "reviews.rating"]]
data_set.columns = ["reviews", "score"]
data_set

,reviews,score
0,This product so far has not disappointed. My c...,5.0
1,great for beginner or experienced person. Boug...,5.0
2,Inexpensive tablet for him to use and learn on...,5.0
3,I've had my Fire HD 8 two weeks now and I love...,4.0
4,I bought this for my grand daughter when she c...,5.0
...,...,...
34655,This is not appreciably faster than any other ...,3.0
34656,Amazon should include this charger with the Ki...,1.0
34657,Love my Kindle Fire but I am really disappoint...,1.0
34658,I was surprised to find it did not come with a...,1.0


In [72]:
data_set["sentiment"] = np.where(data_set["score"] >= 4, 1, -1)
data_set


C:\Users\91971\AppData\Local\Temp\ipykernel_9632\2428003657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set["sentiment"] = np.where(data_set["score"] >= 4, 1, -1)


,reviews,score,sentiment
0,This product so far has not disappointed. My c...,5.0,1
1,great for beginner or experienced person. Boug...,5.0,1
2,Inexpensive tablet for him to use and learn on...,5.0,1
3,I've had my Fire HD 8 two weeks now and I love...,4.0,1
4,I bought this for my grand daughter when she c...,5.0,1
...,...,...,...
34655,This is not appreciably faster than any other ...,3.0,-1
34656,Amazon should include this charger with the Ki...,1.0,-1
34657,Love my Kindle Fire but I am really disappoint...,1.0,-1
34658,I was surprised to find it did not come with a...,1.0,-1


In [94]:
df_majority = data_set[data_set['sentiment'] == 1]
df_minority = data_set[data_set['sentiment'] == -1]


In [98]:
print(len(df_majority))
print(len(df_minority))


32316
2344


In [102]:
nltk.download('punkt_tab')
# Balance the dataset
df_majority = data_set[data_set['sentiment'] == 1]
df_minority = data_set[data_set['sentiment'] == -1]

# Downsample majority class
df_majority_downsampled = resample(df_majority,n_samples=len(df_minority),random_state=42)



# Combine balanced dataset
balanced_data = pd.concat([df_majority_downsampled, df_minority])

# Preprocess all reviews
balanced_data['processed_reviews'] = balanced_data['reviews'].apply(preprocess_text)


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\91971\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [22]:
# Split into train and test sets
train, test = train_test_split(balanced_data, test_size=0.3, random_state=42)

In [134]:
# Save to CSV
train[['processed_reviews', 'sentiment']].to_csv('train.csv', index=False,header=False)
test[['processed_reviews', 'sentiment']].to_csv('test.csv', index=False,header=False)


In [136]:
# Read training data
with open("train.csv", 'r') as file:
    reviews = list(csv.reader(file))

In [106]:
# Get positive and negative texts
def get_text(reviews, score):
    return " ".join([r[0] for r in reviews if r[1] == str(score)])

negative_text = get_text(reviews, -1)
positive_text = get_text(reviews, 1)

In [112]:
# Count words
def count_text(text):
    words = re.split("\s+", text)
    return Counter(words)

negative_counts = count_text(negative_text)
positive_counts = count_text(positive_text)

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\91971\AppData\Local\Temp\ipykernel_9632\24970055.py:3: SyntaxWarning: invalid escape sequence '\s'
  words = re.split("\s+", text)


In [120]:
# Get class counts
def get_y_count(score):
    return len([r for r in reviews if r[1] == str(score)])

positive_review_count = get_y_count(1)
negative_review_count = get_y_count(-1)

# Class probabilities
prob_positive = positive_review_count / len(reviews)
prob_negative = negative_review_count / len(reviews)


In [122]:
print(prob_positive)
print(prob_negative)

0.49832368180432796
0.501676318195672


In [124]:
# Improved prediction function with log probabilities
def make_class_prediction(text, counts, class_prob, class_count):
    log_prediction = math.log(class_prob) 
    text_counts = Counter(re.split("\s+", text))
    total_words = sum(counts.values()) + class_count
    
    for word in text_counts:
        word_prob = (counts.get(word, 0) + 1) / total_words
        log_prediction += text_counts[word] * math.log(word_prob)
    
    return log_prediction


<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
C:\Users\91971\AppData\Local\Temp\ipykernel_9632\2009404789.py:4: SyntaxWarning: invalid escape sequence '\s'
  text_counts = Counter(re.split("\s+", text))


In [126]:
def evaluate_model(test_data):
    actual = []
    predictions = []
    
    for review in test_data:
        if len(review) < 2:  # Skip malformed rows
            continue
            
        text = review[0]
        try:
            true_label = int(review[1])
        except ValueError:
            continue  # Skip header row if it exists
            
        actual.append(true_label)
        
        neg_pred = make_class_prediction(text, negative_counts, prob_negative, negative_review_count)
        pos_pred = make_class_prediction(text, positive_counts, prob_positive, positive_review_count)
        
        pred = -1 if neg_pred > pos_pred else 1
        predictions.append(pred)
    
    if not actual:  # Check if we have any valid data
        print("No valid test data found")
        return
    
    print("\nClassification Report:")
    print(classification_report(actual, predictions, target_names=['Negative', 'Positive']))
    
    print("\nConfusion Matrix:")
    print(confusion_matrix(actual, predictions))
    
    fpr, tpr, thresholds = roc_curve(actual, predictions, pos_label=1)
    print(f"\nAUC: {auc(fpr, tpr):.2f}")

In [128]:
# Load test data properly
test_data = []
with open("test.csv", 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if len(row) >= 2:  # Only take rows with both text and label
            test_data.append(row)


In [130]:
# Evaluate model
evaluate_model(test_data)



Classification Report:
              precision    recall  f1-score   support

    Negative       0.81      0.73      0.77       698
    Positive       0.76      0.83      0.79       709

    accuracy                           0.78      1407
   macro avg       0.78      0.78      0.78      1407
weighted avg       0.78      0.78      0.78      1407


Confusion Matrix:
[[508 190]
 [118 591]]

AUC: 0.78


In [132]:
# Interactive prediction
def predict_sentiment():
    while True:
        review_text = input("\nEnter a review to analyze (or 'quit' to exit): ")
        if review_text.lower() == 'quit':
            break
        
        processed_text = preprocess_text(review_text)
        neg_pred = make_class_prediction(processed_text, negative_counts, prob_negative, negative_review_count)
        pos_pred = make_class_prediction(processed_text, positive_counts, prob_positive, positive_review_count)
        
        print("\nPredicted sentiment:", "POSITIVE" if pos_pred > neg_pred else "NEGATIVE")

# Run interactive prediction
print("\nReview Sentiment Analysis Tool")
predict_sentiment()


Review Sentiment Analysis Tool



Enter a review to analyze (or 'quit' to exit):  i love this product



Predicted sentiment: POSITIVE



Enter a review to analyze (or 'quit' to exit):  i hated it



Predicted sentiment: NEGATIVE


...


Enter a review to analyze (or 'quit' to exit):  extremely disappointed



Predicted sentiment: NEGATIVE



Enter a review to analyze (or 'quit' to exit):  extremely loved it



Predicted sentiment: POSITIVE



Enter a review to analyze (or 'quit' to exit):  quit
